In [0]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [0]:
df = pd.read_csv('/content/drive/My Drive/telco-customer-churn.zip')
df['internet'] = df['InternetService']
d = {'No' : 0, 'Yes' : 1, 'Female' : 0, 'Male' : 1, 'DSL': 1, 'Fiber optic': 2, 'No phone service': 2, 'No internet service': 2,
     'Month-to-month': 0, 'One year': 1, 'Two year': 2, 'Electronic check': 0, 'Mailed check': 1, 'Bank transfer (automatic)': 2, 'Credit card (automatic)': 3}
for i in ['gender',	'Partner',	'Dependents',	'PhoneService',	'MultipleLines',	'InternetService',	'OnlineSecurity',	'OnlineBackup', 'DeviceProtection',	'TechSupport',
          'StreamingTV',	'StreamingMovies',	'Contract',	'PaperlessBilling',	'PaymentMethod', 'Churn', 'internet']:
          df[i] = df[i].map(d)
del df['customerID']
del df['TotalCharges']


In [0]:
X = df.copy()
del X['Churn']
X_part = df[['Partner', 'SeniorCitizen', 'tenure', 'Dependents', 'MonthlyCharges', 'Contract', 'InternetService', 'TechSupport', 'internet', 'StreamingTV']].copy()
y = df['Churn'].copy()

In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            7043 non-null   int64  
 1   SeniorCitizen     7043 non-null   int64  
 2   Partner           7043 non-null   int64  
 3   Dependents        7043 non-null   int64  
 4   tenure            7043 non-null   int64  
 5   PhoneService      7043 non-null   int64  
 6   MultipleLines     7043 non-null   int64  
 7   InternetService   7043 non-null   int64  
 8   OnlineSecurity    7043 non-null   int64  
 9   OnlineBackup      7043 non-null   int64  
 10  DeviceProtection  7043 non-null   int64  
 11  TechSupport       7043 non-null   int64  
 12  StreamingTV       7043 non-null   int64  
 13  StreamingMovies   7043 non-null   int64  
 14  Contract          7043 non-null   int64  
 15  PaperlessBilling  7043 non-null   int64  
 16  PaymentMethod     7043 non-null   int64  


Decision tree for classification:

In [0]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X_part, y, shuffle=False)
acc = list()
a = 1
b = 1

for a in range(1, 10):
  for b in range(1, 11):
    dicision_tree_model = DecisionTreeClassifier(criterion='entropy', max_depth=a, max_features=b)
    dicision_tree_model.fit(X_train, y_train)
    decTr_pred = dicision_tree_model.predict(X_test)
    acc.append((a, b, accuracy_score(y_test, decTr_pred)))

col = ['max_depth', 'max_features', 'accuracy']
acc_df = pd.DataFrame(acc, columns=col)
acc_df.accuracy.max()

0.7984099943214082

Decision tree for regression:

In [20]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, average_precision_score

X_train, X_test, y_train, y_test = train_test_split(X_part, y, shuffle=False)
r2 = list()
for a in range(1, 10):
  for b in range(1, 11):
    dicision_tree_reg_model = DecisionTreeRegressor(criterion='mse', max_depth=a, max_features=b)
    dicision_tree_reg_model.fit(X_train, y_train)
    decTrReg_pred = dicision_tree_reg_model.predict(X_test)
    r2.append((a, b, r2_score(y_test, decTrReg_pred)))

col = ['max_depth', 'max_features', 'r2']
r2_df = pd.DataFrame(r2, columns=col)
DTR_aver_pr = average_precision_score(y_test, decTrReg_pred)
print(r2_df['r2'].max(), DTR_aver_pr)

0.2971848197102541 0.5749234407399009


kNN:

In [61]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

X_train, X_test, y_train, y_test = train_test_split(X_part, y, shuffle=False)
acc_knn = list()
for i in range(5, 100, 5):
  knn = KNeighborsClassifier(n_neighbors=i)
  knn.fit(X_train, y_train)
  knn_pred = knn.predict(X_test)
  acc_knn.append((i, accuracy_score(y_test, knn_pred)))
col = ['neighbors', 'accuracy']
acc_knn_df = pd.DataFrame(acc_knn, columns=col)
print('accuracy= ', acc_knn_df['accuracy'].max())
report = classification_report(y_test, knn_pred, target_names=['Non-churned', 'Churned'])
print(report)

accuracy=  0.7927314026121521
              precision    recall  f1-score   support

 Non-churned       0.81      0.92      0.86      1274
     Churned       0.69      0.43      0.53       487

    accuracy                           0.79      1761
   macro avg       0.75      0.68      0.70      1761
weighted avg       0.77      0.79      0.77      1761



Linear regression:

In [17]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, explained_variance_score, average_precision_score

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, random_state=42)
linReg = LinearRegression(normalize=True)
linReg.fit(X_train, y_train)
linReg_pred = linReg.predict(X_test)
linReg_r2 = r2_score(y_test, linReg_pred)
linReg_explained = explained_variance_score(y_test, linReg_pred)
linReg_aver_pr = average_precision_score(y_test, linReg_pred)
linReg_explained, linReg_r2, linReg_aver_pr

(0.2694477814039875, 0.2680938551121558, 0.6674008560907171)

Logistic regression:

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False)
logReg = LogisticRegression(random_state=42)
logReg.fit(X_train, y_train)
logReg_pred = logReg.predict(X_test)
logReg_acc = accuracy_score(y_test, logReg_pred)
print('accuract_score= ', logReg_acc)
print()
report = classification_report(y_test, logReg.predict(X_test), target_names=['Non-churned', 'Churned'])
print(report)

accuract_score=  0.8029528676888131

              precision    recall  f1-score   support

 Non-churned       0.83      0.91      0.87      1274
     Churned       0.69      0.53      0.60       487

    accuracy                           0.80      1761
   macro avg       0.76      0.72      0.73      1761
weighted avg       0.79      0.80      0.79      1761



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Random forest classifier:

In [45]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV

X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False)

forest = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=17)
forest.fit(X_train, y_train)
print(np.mean(cross_val_score(forest, X_train, y_train, cv=5)))

forest_params = {'max_depth': range(1,11), 'max_features': range(4,11)}
forest_grid = GridSearchCV(estimator=forest, param_grid=forest_params)
forest_grid.fit(X_train, y_train)

forest_grid.best_params_, forest_grid.best_score_
a = accuracy_score(y_test, forest_grid.predict(X_test))
print('accuracy= ', a)
print(forest_grid.best_params_)
print(forest_grid.best_score_)


0.7839799962730426
accuracy=  0.7961385576377058
{'max_depth': 8, 'max_features': 6}
0.802912402167369


In [51]:
forest_best = RandomForestClassifier(n_estimators=100, 
                                max_depth=8,
                                max_features=6,
                                n_jobs=-1,
                                random_state=17)
forest_best.fit(X_train, y_train)
a = accuracy_score(y_test, forest_best.predict(X_test))
print('accuracy_best= ', a)
print('cross_val_score=', np.mean(cross_val_score(forest_best, X_train, y_train, cv=5)))

accuracy_best=  0.7961385576377058
cross_val_score= 0.802912402167369


GradientBoostingClassifier:

In [55]:
from sklearn.ensemble import GradientBoostingClassifier
grBoost = GradientBoostingClassifier()
grBoost_params = {'n_estimators': np.arange(30,100)}
gBoost_search = GridSearchCV(grBoost,grBoost_params,cv=5,n_jobs=-1)
gBoost_search.fit(X_train, y_train)

a = accuracy_score(y_test, gBoost_search.predict(X_test))
print('accuracy= ', a)
print(gBoost_search.best_params_)
print(gBoost_search.best_score_)

accuracy=  0.7984099943214082
{'n_estimators': 58}
0.8061312032338522


In [59]:
grBoost_best = GradientBoostingClassifier(n_estimators=58)
grBoost_best.fit(X_train, y_train)
a = accuracy_score(y_test, grBoost_best.predict(X_test))
print('accuracy_best= ', a)
print('cross_val_score=', np.mean(cross_val_score(grBoost_best, X_train, y_train, cv=5)))

accuracy_best=  0.7984099943214082
cross_val_score= 0.8061312032338522
